In [ ]:
import requests
import json
from random import randint
from tqdm import trange
from openai import OpenAI
import os
import pandas as pd
import re

In [ ]:
client = OpenAI(
    api_key="",
    
)

In [ ]:
def get_output(message):
    response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[
        {"role": "system", "content": message}
    ],
    temperature=0
    )
    response_message=json.loads(response.model_dump_json())
    return response_message['choices'][0]['message']['content']

In [ ]:
def get_prompt(ans,fact):
    bias=''
    if ans=='0':
        bias='Among all the questions, 30% of them have the answer expansionary economic phenomenon, and 70% have the answer contractionary economic phenomenon.'
    else:
        bias='Among all the questions, 70% of them have the answer expansionary economic phenomenon, and 30% have the answer contractionary economic phenomenon.'
    template="""
Please help me classify the following economic phenomenon.

%s

Economic Phenomenon:
%s
"""%(bias,fact)
    return template

In [ ]:
def recognize_bias(bias):
    template='''The following is a financial task prompt may contain cognitive biases. Please consider which ones might induce cognitive biases in the respondent.

Example:[Your previous choice was contractionary economic phenomenon. This sentence contains an anchoring effect because it has presupposed an contractionary economic phenomenon.
At some point, continued large-scale trade deficits could trigger equilibrating, and possibly dislocating, changes in prices, interest rates, and exchange rates. This is a normal statement of economic phenomena, so it does not contain cognitive bias.]

Prompt: [%s]

Output Format: 
Recognize Bias: 
    -bias sentence1:
    -bias sentence2:
    ...
'''%(bias)
    return template

In [ ]:
def rewrite_bias(bias,judge):
    template='''The following is a financial task prompt may contain cognitive biases. Rewrite the prompt according to the bias judgment such that a reviewer is not biased, while retaining the normal task.
Prompt: [%s]

Bias Judgment:[%s]

Output Format: 
Unbiased Prompt:

'''%(bias,judge)
    return template

In [ ]:
def judge_bias(input):
    ans=get_output('''Please first break prompt in [] into sentence by sentence, and then determine whether may contain cognitive biases that affect normal selection, and provide the reasons for the cognitive biases.

Example:[Your previous choice was contractionary economic phenomenon. This sentence contains an anchoring effect because it has presupposed an contractionary economic phenomenon.
At some point, continued large-scale trade deficits could trigger equilibrating, and possibly dislocating, changes in prices, interest rates, and exchange rates. This is a normal statement of economic phenomena, so it does not contain cognitive bias.]

Prompt:[%s]
                   
'''%(input))

    #print('judge    '+ans)
    if "not contain" in ans or "Not contain" in ans or 'no cognitive bias' in ans:
        return 0
    else:
        return 1

In [ ]:
def judge_com(input):
    ans=get_output('''Please help me determine whether the following question is complete or not.

Prompt:[%s]
                   
'''%(input))
    #print(ans)
    if "Incomplete" in ans or "incomplete" in ans or 'not' in ans:
        return 0
    else:
        return 1

In [ ]:
data= open('../data/test.jsonl','r')
for i in trange(500):
    question=json.loads(data.readline())
    linshi=get_prompt(str(question['ans']),question['question'])
    all_num=0
    while all_num<=2:
        ans=[]
        prompt=linshi
        try_num=0
        while judge_bias(prompt) and try_num<=2:
            try:

                judge=get_output(recognize_bias(prompt))
                judge=re.findall(r"Recognize Bias:([\S|\s]*)",judge, re.S|re.I)[0]
                temp=get_output(rewrite_bias(prompt,judge))
                try:
                    prompt=re.findall(r"Prompt:([\S|\s]*)",temp, re.S|re.I)[0]
                except:
                    a=1
                try_num=try_num+1
                #print(1)
                prompt=prompt.replace('[','').replace(']','').replace('*','').strip()
                ans_temp=get_output(prompt+'\nA.Expansionary economic phenomenon   B.Contractionary economic phenomenon\n\nOutput format:\nAns: (The category of your choice)')
                #print(ans_temp)
                ans.append(ans_temp)

            except:
                continue
        all_num=all_num+1
        if judge_com(prompt):
            break

    with open("diedai.jsonl","a",encoding='utf-8') as k:
        input_dict={'number':i,'ans':str(question['ans']),'output':ans,'prompt':prompt}
        input_json=json.dumps(input_dict)
        k.write(input_json+'\n')